<a href="https://colab.research.google.com/github/Gr8eye/DS-Unit-1-Sprint-2-Statistics/blob/master/module3/LS_DS_123_Introduction_to_Bayesian_Inference_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science Module 123

## Introduction to Bayesian Inference




## Assignment - Code it up!

We used pure math to apply Bayes Theorem to drug tests. Now write Python code to reproduce the results! This is purposefully open ended - you'll have to think about how you should represent probabilities and events. You can and should look things up.

Specific goals/targets:

### 1) Write a function 

`def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):` 

You should only truly need these two values in order to apply Bayes Theorem. In this example, imagine that individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000. 
 - What is the probability that a person is drunk after one positive breathalyzer test?
 - What is the probability that a person is drunk after two positive breathalyzer tests?
 - How many positive breathalyzer tests are needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

### 2) Explore `scipy.stats.bayes_mvs`  
Read its documentation, and experiment with it on data you've tested in other ways earlier this week.
 - Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. The following are some potential ways that you could visualize confidence intervals on your graph:
  - [Matplotlib Error Bars](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.pyplot.errorbar.html)
  - [Seaborn barplot with error bars](https://seaborn.pydata.org/generated/seaborn.barplot.html)
  - [Vertical ines to show bounds of confidence interval](https://www.simplypsychology.org/confidence-interval.jpg)
  - [Confidence Intervals on Box Plots](https://matplotlib.org/3.1.1/api/_as_gen/matplotlib.axes.Axes.boxplot.html)

### 3) In your own words, summarize the difference between Bayesian and Frequentist statistics

If you're unsure where to start, check out [this blog post of Bayes theorem with Python](https://dataconomy.com/2015/02/introduction-to-bayes-theorem-with-python/).



In [0]:
#Imports
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt

In [36]:
# Bringing in and setting up data for second part...
Cols = ['Party','Handicapped-Infants', 'Water-Projecjt', 'Budget',
        'Physician-Fee-Freeze', 'El-Salvador-Aid', 'Religious Groups',
        'Anti-Satellite-Ban', 'Aid-to-Contras', 'MX-Missile', 'Immigration',
        'Synfuels', 'Education', 'Right-to-Sue', 'Crime', 'Duty-Free', 
        'South-Africa']
! wget https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
HORV84 = pd.read_csv('house-votes-84.data', header=None,
                     names = Cols, na_values="?")
HORV84.replace({'y':1, 'n':0}, inplace=True)
HORV84.head()

--2020-05-27 22:49:09--  https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18171 (18K) [application/x-httpd-php]
Saving to: ‘house-votes-84.data’

house-votes-84.data 100%[===================>]  17.75K  --.-KB/s    in 0.04s   

2020-05-27 22:49:10 (495 KB/s) - ‘house-votes-84.data’ saved [18171/18171]



,Party,Handicapped-Infants,Water-Projecjt,Budget,Physician-Fee-Freeze,El-Salvador-Aid,Religious Groups,Anti-Satellite-Ban,Aid-to-Contras,MX-Missile,Immigration,Synfuels,Education,Right-to-Sue,Crime,Duty-Free,South-Africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


# Write a function
def prob_drunk_given_positive(prob_drunk_prior, false_positive_rate, true_positive_rate):

In this example, individuals are taking a breathalyzer test with an 8% false positive rate, a 100% true positive rate, and that our prior belief about drunk driving in the population is 1/1000.

* the probability that a person is drunk after one positive breathalyzer test
* the probability that a person is drunk after two positive breathalyzer tests
* Number of positive breathalyzer tests needed in order to have a probability that's greater than 95% that a person is drunk beyond the legal limit?

## Bayes Theorem Formula
P(A|B)= (P(A|B)*P(A))/P(B)
* A, B	=	events
* P(A|B)	=	probability of A given B is true
* P(B|A)	=	probability of B given A is true
* P(A), P(B)	=	the independent probabilities of A and B

In [0]:
# Write a function
def BayesT(Probability_Prior, False_Positive_Rate, True_Positive_Rate):
  #Calculate Probability (not Probability_Prior)
  P = 1 - Probability_Prior
  #Calculate Numerator
  Num = True_Positive_Rate*Probability_Prior
  #Calculate Denominator
  Den = ((True_Positive_Rate*Probability_Prior) + (False_Positive_Rate*P))
  #With that information put the formula together for Posterior Probability
  Post_Probability = (Num/Den)
  return Post_Probability

In [9]:
# Use function to test the probability that a person is drunk after one positive breathalyzer test
Test1 = BayesT(1/1000, .08, 1)
Test1

0.012357884330202669

In [29]:
# Use function to test the probability that a person is drunk after two positive breathalyzer tests
Test2 = BayesT(2/1000, .08, 1)
Test2

0.024437927663734114

In [30]:
# How many tests are needed to have a probability that's greater than 95%?
Test3 = BayesT(Test2, .08, 1)
Test3

0.2384586035864174

In [33]:
# It takes four BayesT tests before reaching a probability that's greater than 95%
Test4 = BayesT(Test3,.08, 1)
Test4

0.79650303308355

In [35]:
# 24% of 1000 = 240 
BayesT(240/1000, .08, 1)

0.7978723404255319

In [34]:
Test5 = BayesT(Test4, .08, 1)
Test5

0.9799703468812797

In [28]:
# OR Beyond 650 breathalyzer tests the probability is above 95%
BayesT(650/1000, .08, 1)

0.9587020648967551

# Explore scipy.stats.bayes_mvs

* Create a visualization comparing the results of a Bayesian approach to a traditional/frequentist approach. (with a large sample size they should look close to identical, however, take this opportunity to practice visualizing condfidence intervals in general. 
* Confidence Intervals on Box Plots

# The difference between Bayesian and Frequentist statistics:

## Resources

- [Worked example of Bayes rule calculation](https://en.wikipedia.org/wiki/Bayes'_theorem#Examples) (helpful as it fully breaks out the denominator)
- [Source code for mvsdist in scipy](https://github.com/scipy/scipy/blob/90534919e139d2a81c24bf08341734ff41a3db12/scipy/stats/morestats.py#L139)

## Stretch Goals:

- Go back and study the content from Modules 1 & 2 to make sure that you're really comfortable with them.
- Apply a Bayesian technique to a problem you previously worked (in an assignment or project work) on from a frequentist (standard) perspective
- Check out [PyMC3](https://docs.pymc.io/) (note this goes beyond hypothesis tests into modeling) - read the guides and work through some examples
- Take PyMC3 further - see if you can build something with it!